In [15]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
dataset_path = 'fer2013.csv'
image_size=(48,48)
 
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

/home/sherlock/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [16]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D, Activation, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten, Input, MaxPooling2D, SeparableConv2D
from keras.models import Sequential, Model
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np

In [28]:
#parameters
batch_size = 32
num_epochs = 50
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = '/home/sherlock/Rushi/EmotionRecognition'
l2_regularization = 0.01

In [18]:
data_generator = ImageDataGenerator(featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=.1,
                                   horizontal_flip=True)

In [19]:
regularization = l2(l2_regularization)

In [20]:
#base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer = regularization,
          use_bias = False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer = regularization,
          use_bias = False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

In [21]:
#module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16,(3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16,(3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

In [22]:
#module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32,(3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

In [23]:
#module 3
residual = Conv2D(64, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64,(3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])

In [24]:
#module 4
residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128,(3, 3), padding = 'same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding = 'same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax', name = 'predictions')(x)

In [25]:
model = Model(img_input, output)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


In [33]:
#callbacks
log_file_path = base_path + 'emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [34]:
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

Epoch 1/50
898/897 [==============================] - 258s 287ms/step - loss: 1.7601 - acc: 0.3375 - val_loss: 1.8866 - val_acc: 0.2999

Epoch 00001: val_loss improved from inf to 1.88665, saving model to /home/sherlock/Rushi/EmotionRecognition_mini_XCEPTION.01-0.30.hdf5
Epoch 2/50
898/897 [==============================] - 254s 282ms/step - loss: 1.5183 - acc: 0.4285 - val_loss: 1.6117 - val_acc: 0.3810

Epoch 00002: val_loss improved from 1.88665 to 1.61167, saving model to /home/sherlock/Rushi/EmotionRecognition_mini_XCEPTION.02-0.38.hdf5
Epoch 3/50
898/897 [==============================] - 222s 247ms/step - loss: 1.4230 - acc: 0.4696 - val_loss: 1.3543 - val_acc: 0.4974

Epoch 00003: val_loss improved from 1.61167 to 1.35432, saving model to /home/sherlock/Rushi/EmotionRecognition_mini_XCEPTION.03-0.50.hdf5
Epoch 4/50
898/897 [==============================] - 190s 212ms/step - loss: 1.3453 - acc: 0.4959 - val_loss: 1.5183 - val_acc: 0.4380

Epoch 00004: val_loss did not improve f

In [3]:
!pip3 install --user imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp36-none-any.whl size=26734 sha256=c752df11f61ea647ba2c63438759ec58ed73a2f5b4dbf6ad08e283e022373d79
  Stored in directory: /home/sherlock/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [4]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import cv2
import numpy as np
import sys

In [5]:
detection_model_path = 'haarcascade_frontalface_default.xml'
emotion_model_path = 'EmotionRecognition_mini_XCEPTION.44-0.61.hdf5'
img_path = sys.argv[1]

In [7]:
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile = False)
EMOTIONS = ["angry","disgust", "scared", "happy", "sad", "surprised", "neutral"]

In [9]:
#Reading the frame
orig_frame = cv2.imread(img_path)
frame = cv2.imread(img_path, 0)
faces = face_detection.detectMultiScale(frame, scaleFactor=1.1,
                                          minNeighbors = 5, minSize = (30, 30),
                                          flags = cv2.CASCADE_SCALE_IMAGE)

In [15]:
if len(faces) > 0:
    faces = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
    (fX, fY, fW, fH) = faces
    roi = frame[fY:fY + fH, fX:fX + fW]
    roi = cv2.resize(roi, (48,48))
    roi = roi.astype("float")/ 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)
    preds = emotion_classifier.predict(roi)[0]
    emotion_probability = np.max(preds)
    label = EMOTIONS[preds.argmax()]
    cv2.putText(orig_frame, label, (fX, fY-10), cv2.FONT_HASHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.rectangle(orig_frame, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)
    
cv2.imshow('test_face', orig_frame)
cv2.imwrite('test_output/' + img_path.split('/')[-1], orig_frame)
if (cv2.waitKey(2000) & 0xFF == ord('q')):
    sus.exit("Thanks")
cv2.destroyAllWindows()

error: OpenCV(3.4.4) /io/opencv/modules/highgui/src/window.cpp:356: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
